# Input

In [6]:
type_chart = '''
Normal:   X X X X X X X X X X X X R I X X R X
Fire:     X R R S X S X X X X X S R X R X S X
Water:    X S R R X X X X S X X X S X R X X X 
Grass:    X R S R X X X R S R X R S X R X R X
Electric: X X S R R X X X I S X X X X R X X X
Ice:      X R R S X R X X S S X X X X S X R X
Fighting: S X X X X S X R X R R R S I X S S R
Poison:   X X X S X X X R R X X X R R X X I S
Ground:   X S X R S X X S X I X R S X X X S X
Flying:   X X X S R X S X X X X S R X X X R X
Psychic:  X X X X X X S S X X R X X X X I R S
Bug:      X R X S X X R R X R S X X R X S R R
Rock:     X S X X X S R X R S X S X X X X R X
Ghost:    I X X X X X X X X X S X X S X R X X
Dragon:   X X X X X X X X X X X X X X S X R I
Dark:     X X X X X X R X X X S X X S X R X R
Steel:    X R R X R S X X X X X X S X X X R S
Fariy:    X R X X X X S R X X X X X X S S R X
'''
   

In [5]:
print(type_chart.replace('X',' '))


Normal:                           R I     R  
Fire:       R R S   S           S R   R   S  
Water:      S R R         S       S   R       
Grass:      R S R       R S R   R S   R   R  
Electric:     S R R       I S         R      
Ice:        R R S   R     S S         S   R  
Fighting: S         S   R   R R R S I   S S R
Poison:         S       R R       R R     I S
Ground:     S   R S     S   I   R S       S  
Flying:         S R   S         S R       R  
Psychic:              S S     R         I R S
Bug:        R   S     R R   R S     R   S R R
Rock:       S       S R   R S   S         R  
Ghost:    I                   S     S   R    
Dragon:                               S   R I
Dark:                 R       S     S   R   R
Steel:      R R   R S             S       R S
Fariy:      R         S R             S S R  

